# JSON VALUES

In [8]:
import firebase_admin
from firebase_admin import credentials, firestore
import datetime
import json
import pytz  # make sure you have pytz installed

if not firebase_admin._apps:
    cred = credentials.Certificate("track-pit-firebase-adminsdk-fbsvc-989e750375.json")
    firebase_admin.initialize_app(cred)

db = firestore.client()

# Malaysia timezone
MALAYSIA_TZ = pytz.timezone("Asia/Kuala_Lumpur")


def serialize_value(value):
    """Convert Firestore types into JSON-safe Python types."""
    from google.cloud.firestore_v1._helpers import DatetimeWithNanoseconds

    if isinstance(value, bytes):
        return value.decode("utf-8", errors="ignore")

    if isinstance(value, (datetime.datetime, DatetimeWithNanoseconds)):
        # Ensure timezone-aware datetime
        if value.tzinfo is None:
            value = value.replace(tzinfo=datetime.timezone.utc)
        # Convert to Malaysia timezone
        local_dt = value.astimezone(MALAYSIA_TZ)
        # Format to readable string
        return local_dt.strftime("%d %b %Y, %I:%M %p")  # Example: "20 Sep 2025, 01:45 AM"

    if isinstance(value, dict):
        return {k: serialize_value(v) for k, v in value.items()}

    if isinstance(value, list):
        return [serialize_value(v) for v in value]

    return value


def fetch_collection(path, sample_limit=None):
    """
    Recursively fetch all documents & subcollections under a collection path.
    Returns nested dict.
    """
    result = {}
    ref = db.collection(path)
    docs = ref.stream() if sample_limit is None else ref.limit(sample_limit).stream()

    for doc in docs:
        doc_data = {k: serialize_value(v) for k, v in doc.to_dict().items()}

        # Handle subcollections
        sub_data = {}
        for subcol in ref.document(doc.id).collections():
            sub_data[subcol.id] = fetch_collection(f"{path}/{doc.id}/{subcol.id}")

        if sub_data:
            doc_data["_subcollections"] = sub_data

        result[doc.id] = doc_data

    return result


# --- Dump specific top-level collections ---
all_data = {
    "users": fetch_collection("users"),
    # "plates": fetch_collection("plates"),
    # "feedback": fetch_collection("feedback"),
    # "faqs": fetch_collection("faqs"),
}

print(json.dumps(all_data, indent=2, ensure_ascii=False))

{
  "users": {
    "4nI4UHmPifPLkwGsWu0zJDy68oE2": {
      "points": 0,
      "email": "a@a.com",
      "updatedAt": "18 Sep 2025, 07:23 PM",
      "fullName": "aaa",
      "createdAt": "18 Sep 2025, 07:23 PM"
    },
    "BFt4LbQPViQbya9Hh46xMG912Qr1": {
      "points": 0,
      "email": "yylim-wm22@student.tarc.edu.my",
      "updatedAt": "20 Sep 2025, 01:22 AM",
      "createdAt": "20 Sep 2025, 01:22 AM",
      "fullName": "YUET YANG LIM"
    },
    "RHok1nfhGBSFuacubmZMer22tKp1": {
      "fullName": "Lim Yuet Yang",
      "email": "a@b.com",
      "updatedAt": "18 Sep 2025, 07:32 PM",
      "points": 0,
      "createdAt": "18 Sep 2025, 07:32 PM"
    },
    "iiYYy1LD2TYMASxgaW1VzTvRkwe2": {
      "points": 0,
      "email": "yigallim@gmail.com",
      "updatedAt": "20 Sep 2025, 01:22 AM",
      "createdAt": "20 Sep 2025, 01:22 AM",
      "fullName": "Ye Yang",
      "_subcollections": {
        "services": {
          "5Zg1spiwEn6lK6om3sjR": {
            "workshopId": 2,
           

# DESC SHAPE

In [3]:
import firebase_admin
from firebase_admin import credentials, firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("track-pit-firebase-adminsdk-fbsvc-989e750375.json")
    firebase_admin.initialize_app(cred)

db = firestore.client()


def detect_type(value):
    if isinstance(value, str):
        return "string"
    elif isinstance(value, bool):
        return "bool"
    elif isinstance(value, int):
        return "int"
    elif isinstance(value, float):
        return "float"
    elif isinstance(value, list):
        return "list"
    elif isinstance(value, dict):
        return "map"
    elif hasattr(value, "timestamp"):  # Firestore timestamp
        return "timestamp"
    else:
        return type(value).__name__

def describe_collection(path, depth=0, sample_limit=5):
    indent = "  " * depth
    try:
        docs = db.collection(path).limit(sample_limit).stream()
    except Exception as e:
        print(f"{indent}⚠️ Cannot access {path}: {e}")
        return

    print(f"{indent}Collection: {path}")
    for doc in docs:
        print(f"{indent}  Document ID: {doc.id}")
        data = doc.to_dict()
        for key, val in data.items():
            print(f"{indent}    - {key}: {detect_type(val)}")
        
        # Recurse into subcollections
        subcollections = db.collection(path).document(doc.id).collections()
        for subcol in subcollections:
            describe_collection(f"{path}/{doc.id}/{subcol.id}", depth+2, sample_limit)


# Example: top-level collections
describe_collection("users")       # will recurse into vehicles, bookings, serviceRecords
describe_collection("plates")
describe_collection("feedback")
describe_collection("faqs")

Collection: users
  Document ID: 4nI4UHmPifPLkwGsWu0zJDy68oE2
    - points: int
    - email: string
    - updatedAt: timestamp
    - fullName: string
    - createdAt: timestamp
  Document ID: BFt4LbQPViQbya9Hh46xMG912Qr1
    - points: int
    - email: string
    - updatedAt: timestamp
    - createdAt: timestamp
    - fullName: string
  Document ID: RHok1nfhGBSFuacubmZMer22tKp1
    - fullName: string
    - email: string
    - updatedAt: timestamp
    - points: int
    - createdAt: timestamp
  Document ID: iiYYy1LD2TYMASxgaW1VzTvRkwe2
    - points: int
    - email: string
    - updatedAt: timestamp
    - createdAt: timestamp
    - fullName: string
    Collection: users/iiYYy1LD2TYMASxgaW1VzTvRkwe2/services
      Document ID: 3867E755Y9HuFaTeAFLQ
        - workshopId: int
        - notes: string
        - updatedAt: timestamp
        - bookedDateTime: timestamp
        - vehicleId: string
        - serviceTypeId: int
        - createdAt: timestamp
        Collection: users/iiYYy1LD2TYMASx

# FIREBASE SCHEMA

### `users/{uid}`

* **Fields**

  * `email` (string)
  * `fullName` (string)
  * `points` (int)
  * `vehicleCount` (int, optional)
  * `hasCompletedVehicleSetup` (bool, optional)
  * `createdAt` (timestamp)
  * `updatedAt` (timestamp)

* **Subcollections**

  * **`vehicles/{vehicleId}`**

    * `plateNumber` (string)
    * `model` (string)
    * `year` (int)
    * `chassisNumber` (string)
    * `mileage` (int)
    * `transmission` (string)
    * `createdAt`, `updatedAt` (timestamp)

  * **`services/{serviceId}`**

    * `vehicleId` (string)
    * `serviceTypeId` (int)
    * `workshopId` (int)
    * `bookedDateTime` (timestamp)
    * `notes` (string)
    * `createdAt`, `updatedAt` (timestamp)

  * **`invoices/{invoiceId}`**

    * `vehicleId` (string → reference to `vehicles/{vehicleId}`)
    * `serviceId` (string → reference to `services/{serviceId}`)
    * `workshopId` (int)
    * `price` (double)
    * `paid` (bool)
    * `issuedAt` (timestamp)
    * `createdAt`, `updatedAt` (timestamp)

  * **`payments/{paymentId}`**

    * `invoiceIds` (array of strings)
    * `subtotal` (double)
    * `discount` (double)
    * `netTotal` (double)
    * `paidAt` (timestamp)

---

### `plates/{plate}`

* `uid` (string)
* `vehicleId` (string)
* `createdAt` (timestamp)

---

### `feedback/{feedbackId}`

* `uid` (string)
* `serviceId` (string)
* `email` (string)
* `rating` (int)
* `message` (string)
* `createdAt` (timestamp)
* `updatedAt` (timestamp)

---

### `faqs/{faqId}`

* `question` (string)
* `answer` (string)
* `category` (string)
